In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

In [ ]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

ecr_repository = 'docker_R'
uri_suffix = 'amazonaws.com'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

print('account_id: ',account_id)
print('region: ',region)
print('processing_repository_uri: ',processing_repository_uri)

In [ ]:
role = get_execution_role()
session = sagemaker.Session()
s3_output = session.default_bucket()
s3_prefix = 'demo'
s3_source = 'sagemaker-us-east-1-nnn'

subir los script de R a s3

In [ ]:
code_s3_prefix = '{}/code'.format(s3_prefix)
code_s3 = session.upload_data(path='/home/ec2-user/SageMaker/proyecto', key_prefix=code_s3_prefix)
print(code_s3)

In [ ]:
rawdata_s3_prefix = '{}/data/files'.format(s3_prefix)
raw_s3="s3://sagemaker-us-east-1-nn/demo/data/files"

In [ ]:
script_processor = ScriptProcessor(command=['Rscript'],
                image_uri=processing_repository_uri,
                role=role,
                instance_count=1,
                instance_type='ml.c5.2xlarge')

job

In [ ]:
processing_job_name = "demo-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination = 's3://{}/{}/data'.format(s3_output, s3_prefix)
 
print(output_destination)
script_processor.run(code='/home/ec2-user/SageMaker/proyecto/scripts.R',
                      job_name=processing_job_name,
                      inputs=[ProcessingInput(
                                    source=raw_s3,
                                    destination='/opt/ml/processing/input'),
                              ProcessingInput(
                                    source=code_s3,
                                    destination='/opt/ml/processing/code')],
                      outputs=[ProcessingOutput(output_name='csv',
                                                destination='{}/csv'.format(output_destination),
                                                source='/opt/ml/processing/files')
                              ])

preprocessing_job_description = script_processor.jobs[-1].describe()